In [8]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [9]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Pouembout,NC,2021-10-07 12:14:15,-21.1333,164.9000,67.41,73,2,7.18
1,1,Qaanaaq,GL,2021-10-07 12:14:15,77.4840,-69.3632,18.37,57,42,7.96
2,2,Mogadishu,SO,2021-10-07 12:14:16,2.0371,45.3438,80.42,69,9,18.77
3,3,Necochea,AR,2021-10-07 12:14:16,-38.5473,-58.7368,48.04,72,3,7.74
4,4,Jamestown,US,2021-10-07 12:12:33,42.0970,-79.2353,63.72,100,90,5.75


In [10]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [11]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [12]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]

temps = []
for temp in max_temp:
    temps.append(max(temp, 0))
    
# Assign the figure variable.
fig = gmaps.figure()

# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp])
#heat_layer = gmaps.heatmap_layer(locations, weights=temps)

# Add the heatmap layer.
fig.add_layer(heat_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [13]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [14]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [15]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
2,2,Mogadishu,SO,2021-10-07 12:14:16,2.0371,45.3438,80.42,69,9,18.77
5,5,Atuona,PF,2021-10-07 12:14:16,-9.8000,-139.0333,77.38,77,8,14.92
10,10,Hermanus,ZA,2021-10-07 12:14:18,-34.4187,19.2345,79.90,55,38,1.63
11,11,Cape Town,ZA,2021-10-07 12:12:48,-33.9258,18.4232,80.11,7,4,17.09
17,17,Kapaa,US,2021-10-07 12:14:20,22.0752,-159.3190,75.52,80,20,1.99
23,23,Oriximina,BR,2021-10-07 12:13:07,-1.7656,-55.8661,87.53,54,76,3.04
26,26,Ponta Delgada,PT,2021-10-07 12:14:23,37.7333,-25.6667,76.50,88,75,11.50
30,30,Nouadhibou,MR,2021-10-07 12:14:25,20.9310,-17.0347,79.75,50,77,11.97
33,33,Saint-Philippe,RE,2021-10-07 12:14:26,-21.3585,55.7679,80.15,82,60,11.50
35,35,Aguilas,ES,2021-10-07 12:14:26,37.4063,-1.5829,87.21,64,4,4.94


In [16]:
preferred_cities_df.count()

City_ID       176
City          176
Country       176
Date          176
Lat           176
Lng           176
Max Temp      176
Humidity      176
Cloudiness    176
Wind Speed    176
dtype: int64

In [17]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Mogadishu,SO,80.42,2.0371,45.3438,
5,Atuona,PF,77.38,-9.8000,-139.0333,
10,Hermanus,ZA,79.90,-34.4187,19.2345,
11,Cape Town,ZA,80.11,-33.9258,18.4232,
17,Kapaa,US,75.52,22.0752,-159.3190,
23,Oriximina,BR,87.53,-1.7656,-55.8661,
26,Ponta Delgada,PT,76.50,37.7333,-25.6667,
30,Nouadhibou,MR,79.75,20.9310,-17.0347,
33,Saint-Philippe,RE,80.15,-21.3585,55.7679,
35,Aguilas,ES,87.21,37.4063,-1.5829,


In [23]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    #hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    # Grab the first hotel from the results and store the name.
try:
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
except (IndexError):
    print("Hotel not found... skipping.")

In [24]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [25]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [27]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [28]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [29]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))